### Using Jupyter Notebook and CLI tool for uploading attribute, attribute options and families

- Clone repository to local. Install and run.<code>composer require composer</code>

### A: Jupyter Notebook:-
> 1. Enter input file path in ```inputPath``` variable.
> 2. Enter attribute column names in ```req_columns``` variable. (Default required are:-'3M ID','Marketplace Formal Name', 'Marketplace Description', 'Marketplace Description        Extended', 'Main Picture ', 'Shipper - Item Quantity').
> 3. Enter output file path in ```ouputFileName``` variable.
> 4. Run the whole script from top cell.
> 5. OutputFile will now be generated at provided location.

### B: Terminal commands to generate JSONs and push attribute, attribute options and families.
> 1. To generate JSONs for attribute and families; enter command-
> 
>> ```bin/console app:gen-attrs-json <validInputExcelFilePath> <outputJsonFilePath>```
>
> 2. In order to directly push attributes; add flag --attrPush=1. To push families also; add flag --famPush=1.
>
>> ```bin/console app:gen-attrs-json <validInputExcelFilePath> <outputJsonFilePath> --attrPush=1 --famPush=1```
>
> 3. To generate JSONs for attribute options and families; enter command-
>
>> ```bin/console app:gen-attrs-options-json <validInputExcelFilePath> <outputJsonFilePath>```
>
> 4. In order to directly push attribute; add flag --push=1.
>
>> ```bin/console app:gen-attrs-options-json <validInputExcelFilePath> <outputJsonFilePath> --push=1```


In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import subprocess as sp

#### Setup configurations and file paths
- inputPath :- Input xlsx file path
- outputFileName :- Output xlsx file path.
- req_columns :- Required column names from xlsx file. (Enter exact name. Along with leading or trailing spaces)
- output JSON folder path:- Path to output JSON storage folder. (Two JSONs will be created. One for attributes and family and second for attribute options. Options JSON will have 'Options' in the file name. JSON file name will be taken from out put excel file name.)
- familyName :- Family name for current file.
- attrPush :- Flag to determine whether to push attributes directly from here. (To push enter 1 otherwise 0)
- famPush :- Flag to determine whether to push families directly from here. (To push enter 1 otherwise 0)
- optionsPush :- Flag to determine whether to push options directly from here. (To push enter 1 otherwise 0)

In [28]:
# Required variables
inputPath = "/home/soham/Important/Data/3M/Abrasives/Abrasive Sheets & Rolls/Brush Sheets & Rolls.xlsx"
req_columns = ['3M ID','Marketplace Formal Name', 'Marketplace Description', 'Marketplace Description Extended', 'Main Picture ',
                'Shipper - Item Quantity', #Defaults
                'Abrasive Material','Center Hole Diameter (Imperial)','Grit' ]
outputFileName = '/home/soham/Important/Output/3M/Brush Sheets and Rolls.xlsx'
jsonFolder = '/home/soham/Parser/xml_parser/output/JSON/3M/'
familyName = "Brush Sheets and Rolls"
attrPush = 0
famPush = 0
optionsPush = 0

In [463]:
#Reading excel file
db = pd.read_excel(inputPath, header=None)

In [466]:
# Retriving required columns
df = pd.DataFrame(db)
df2 = df.drop(df.index[0])
headers = df2.iloc[0:1,:]
arr = []
for h in range(len(headers.columns)):
    arr.append(headers[h][1])
df2 = df2.drop(df2.index[0])
df2.columns = arr
df3 = df2.loc[:,req_columns]
df3.head()

,3M ID,Marketplace Formal Name,Marketplace Description,Marketplace Description Extended,Main Picture,Shipper - Item Quantity,Abrasive Material,Center Hole Diameter (Imperial),Grit
2,7100260361,"3M™ Brushlon Sheet 320B, 0.005 3/16 in Trim, G...",3M™ Brushlon Sheet 320B features short-trim ny...,3M™ Brushlon™ Abrasives: Choosing the Right On...,NaN,26.0,Nylon,NaN,Non Pertinent
3,7100260355,"3M™ Brushlon Sheet 320BX, 0.005 1/8 in Trim, B...",3M™ Brushlon Roll 320BX is designed to provide...,3M™ Brushlon™ Abrasives: Choosing the Right On...,NaN,25.0,Nylon,NaN,Non Pertinent
4,7100260360,"3M™ Brushlon Sheet 320B, 0.005 5/32 in Trim, G...",3M™ Brushlon Roll 320BX is designed to provide...,3M™ Brushlon™ Abrasives: Choosing the Right On...,NaN,25.0,Nylon,NaN,Non Pertinent
5,7100260363,"3M™ Brushlon Sheet 320BX, 0.005 5/32 in Trim, ...",3M™ Brushlon Roll 320BX is designed to provide...,3M™ Brushlon™ Abrasives: Choosing the Right On...,NaN,25.0,Nylon,NaN,Non Pertinent
6,7100119912,"3M™ Brushlon Roll 320BX, .005 5/32 in Trim, Bl...",3M™ Brushlon Roll 320B features short-trim nyl...,3M™ Brushlon™ Products: Choosing the Right One...,https://images.salsify.com/image/upload/s--jIO...,NaN,Nylon,NaN,Non Pertinent


In [467]:
#Filetering out invalid columns
invalidValues = ['NaN','0.0 NP','Non Pertinent','']
valCols = []
invalCols = []

for col in df3.columns:
    col_data = df3.loc[:,col]
    isValid = True
    for row in col_data:
        if(row in invalidValues or pd.isna(row)):
            isValid = False
        else:
            isValid = True
            break
    if(isValid==False):
        invalCols.append(col)
    else:
        valCols.append(col)

print("Valid cols are")
for i in valCols:
    print(i)

print("\nInvalid cols are")
for i in invalCols:
    print(i)

Valid cols are
3M ID
Marketplace Formal Name
Marketplace Description
Marketplace Description Extended
Main Picture 
Shipper - Item Quantity
Abrasive Material

Invalid cols are
Center Hole Diameter (Imperial)
Grit


In [468]:
# Filtereing out invalid values from remaining columns
valdf = df3.loc[:, valCols]
valdf = valdf.replace(to_replace=['Non Pertinent','0.0 NP'],value=np.nan)
valdf.head()

,3M ID,Marketplace Formal Name,Marketplace Description,Marketplace Description Extended,Main Picture,Shipper - Item Quantity,Abrasive Material
2,7100260361,"3M™ Brushlon Sheet 320B, 0.005 3/16 in Trim, G...",3M™ Brushlon Sheet 320B features short-trim ny...,3M™ Brushlon™ Abrasives: Choosing the Right On...,NaN,26.0,Nylon
3,7100260355,"3M™ Brushlon Sheet 320BX, 0.005 1/8 in Trim, B...",3M™ Brushlon Roll 320BX is designed to provide...,3M™ Brushlon™ Abrasives: Choosing the Right On...,NaN,25.0,Nylon
4,7100260360,"3M™ Brushlon Sheet 320B, 0.005 5/32 in Trim, G...",3M™ Brushlon Roll 320BX is designed to provide...,3M™ Brushlon™ Abrasives: Choosing the Right On...,NaN,25.0,Nylon
5,7100260363,"3M™ Brushlon Sheet 320BX, 0.005 5/32 in Trim, ...",3M™ Brushlon Roll 320BX is designed to provide...,3M™ Brushlon™ Abrasives: Choosing the Right On...,NaN,25.0,Nylon
6,7100119912,"3M™ Brushlon Roll 320BX, .005 5/32 in Trim, Bl...",3M™ Brushlon Roll 320B features short-trim nyl...,3M™ Brushlon™ Products: Choosing the Right One...,https://images.salsify.com/image/upload/s--jIO...,NaN,Nylon


In [469]:
updtdColName = dict()
updtdColName = {'Marketplace Formal Name':"Name",
                'Marketplace Description':'Short Description',
                'Marketplace Description Extended':'Description',
                'Main Picture ':'Image',
                'Shipper - Item Quantity':'Shipping Quantity'}
newCols = []
for col in valdf.columns:
    if(col in updtdColName.keys()):
        newCols.append(updtdColName[col].strip())
    else:
        col = col.replace("(Imperial)","")
        newCols.append(col.strip())

print(newCols)
valdf.columns = newCols
customCols = ['Brand', 'Manufacturer Name', 'Product Info','URL Key','Specification(s)']
for c in customCols:
    valdf[c] = ""

['3M ID', 'Name', 'Short Description', 'Description', 'Image', 'Shipping Quantity', 'Abrasive Material']


In [470]:
# Final excel structure
valdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 2 to 7
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   3M ID              6 non-null      object
 1   Name               6 non-null      object
 2   Short Description  6 non-null      object
 3   Description        6 non-null      object
 4   Image              2 non-null      object
 5   Shipping Quantity  5 non-null      object
 6   Abrasive Material  6 non-null      object
 7   Brand              6 non-null      object
 8   Manufacturer Name  6 non-null      object
 9   Product Info       6 non-null      object
 10  URL Key            6 non-null      object
 11  Specification(s)   6 non-null      object
dtypes: object(12)
memory usage: 708.0+ bytes


In [471]:
#Creating excel with required columns
valdf.to_excel(outputFileName,index=None)

In [25]:
# Creating json files from jsonFolder and outputFile names.
# familyName = outputFileName.split("/")[-1].split(".")[0].title()
jsonFile = jsonFolder + outputFileName.split("/")[-1].split(".")[0].title().replace(" ","")
jsonAttrFile = jsonFile +".json"
jsonOptionsFile = jsonFile + "Options.json"
jsonOptionsFile

'/home/soham/Parser/xml_parser/output/JSON/3M/BrushSheetsAndRollsOptions.json'

In [24]:
#Executing attribute and options generation and push command from here.
return_list = sp.run(['/home/soham/Parser/xml_parser/bin/console','app:gen-attrs-json',outputFileName, jsonAttrFile, familyName, '--attrPush='+str(attrPush) ,'--famPush='+str(famPush)])
print('GenXlsxAttributeJson.php Execution done. Return code', return_list.returncode)
return_list = sp.run(['/home/soham/Parser/xml_parser/bin/console','app:gen-attrs-options-json',outputFileName, jsonOptionsFile, '--push='+str(optionsPush)])
print('GenAttributeOptionsJson.php Execution done. Return code', return_list.returncode)

PHP Warning:  Module 'apcu' already loaded in Unknown on line 0
PHP Warning:  Module 'apcu' already loaded in Unknown on line 0


Generated json for Name
Generated json for Short Description
Generated json for Description
Generated json for Image
Generated json for Shipping Quantity
Generated json for Abrasive Material
Generated json for Brand
Generated json for Manufacturer Name
Generated json for Product Info
Generated json for URL Key
Generated json for Specification(s)

Setting family label is --> Brush Sheets And Rolls
Setting family code is  --> brush_sheets_and_rolls
GenXlsxAttributeJson.php Execution done. Return code 0
GenAttributeOptionsJson.php Execution done. Return code 0
